In [ ]:
from transformers import TFViTForImageClassification, ViTFeatureExtractor
from PIL import Image
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from google.colab import files




In [ ]:
# --- Load model and feature extractor ---
model_path = '/content/drive/MyDrive/saved_vit_model'
vit_model = TFViTForImageClassification.from_pretrained(model_path)
feature_extractor = ViTFeatureExtractor.from_pretrained(model_path)

All model checkpoint layers were used when initializing TFViTForImageClassification.

All the layers of TFViTForImageClassification were initialized from the model checkpoint at /content/drive/MyDrive/saved_vit_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTForImageClassification for predictions without further training.


In [ ]:
# --- Load product dataset ---
df = pd.read_csv('/content/drive/MyDrive/skincare_recommendations_full.csv')
df.head()


,condition,product,brand,skin_type,category
0,acne,Benzoyl Peroxide 5% Gel,Clean & Clear,oily,cleanser
1,acne,Salicylic Acid Face Wash,Neutrogena,combination,cleanser
2,acne,AcneFree Repair Lotion,AcneFree,oily,moisturizer
3,dry,CeraVe Moisturizing Cream,CeraVe,dry,moisturizer
4,dry,The Ordinary Natural Moisturizing Factors,The Ordinary,dry,moisturizer


In [ ]:
# --- Define category list from your ViT model ---
categories = ['acne', 'Milia', 'Dry', 'Oily', 'Wrinkles', 'Non Wrinkles', 'hyperpigmentation', 'Keratosis', 'Normal']

#  Make sure this is defined BEFORE the function
critical_conditions = ['acne', 'Milia', 'Keratosis', 'hyperpigmentation']

# --- Define recommendation function ---
def recommend_products(condition, df, top_k=3):
    condition = condition.lower()
    matches = df[df['condition'].str.lower() == condition]
    if matches.empty:
        return f"No product found for condition: {condition}"
    return matches[['product', 'brand', 'skin_type', 'category']].head(top_k)



In [ ]:
# --- Predict + Recommend / Refer ---
def predict_image_and_recommend(image_path, model, feature_extractor, df, threshold_high=0.99, threshold_low=0.90):
    image = Image.open(image_path).convert('RGB').resize((224, 224))
    inputs = feature_extractor(images=[np.array(image)], return_tensors='tf')
    logits = model(**inputs).logits
    probs = tf.nn.softmax(logits, axis=1).numpy()[0]

    top_idx = np.argmax(probs)
    confidence = probs[top_idx]
    condition = categories[top_idx]

    print(f"\n📷 Prediction for uploaded image: {image_path}")
    print(f"🧠 Predicted Condition: {condition}")
    print(f"🔢 Confidence: {confidence:.4f}")

    # Decision logic
    if confidence >= threshold_high:
        print("✅ High confidence — recommending products:")
        print(recommend_products(condition, df))
    elif confidence < threshold_low and condition.lower() in [c.lower() for c in critical_conditions]:
        print("⚠️ Low confidence in a critical condition — schedule dermatologist appointment.")
    else:
        print("ℹ️ Confidence not high, but safe to recommend products:")
        print(recommend_products(condition, df))

# --- Upload and run prediction ---
uploaded = files.upload()
for fn in uploaded.keys():
    predict_image_and_recommend(fn, vit_model, feature_extractor, df)

Saving download (6).jpeg to download (6).jpeg

📷 Prediction for uploaded image: download (6).jpeg
🧠 Predicted Condition: Keratosis
🔢 Confidence: 0.7810
⚠️ Low confidence in a critical condition — schedule dermatologist appointment.
